In [1]:
# Async Connector
# Developed by CD
# v2.0.0-prod

from io import StringIO
import time
import numpy as np
import os
from datetime import datetime, timedelta, date
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy import text
from typing import List
from collections import defaultdict, Counter
import pandas as pd
from cryptography.fernet import Fernet
from dotenv import load_dotenv
from io import StringIO
from pathlib import Path
import asyncio
import nest_asyncio
import sys
from typing import Dict, Union, List
nest_asyncio.apply()

if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())


def retrieve_data(queries: List[Dict[str, Union[str, pd.DataFrame, int]]]) -> Dict[str, pd.DataFrame]:
    """
    Retrieve data from Oracle Database (COCC)

    Args:
        queries (List): pass list of queries in specific format
            - List[Dict[str, Union[str, pd.DataFrame, int]]]
    
    Returns:
        data (Dict): Returns a dictionary with df name and the df attached as key/value pair.   

    """
    class DatabaseHandler:
        """
        This class abstracts the connection to the database and allows a clean
        interface for the developer to use.

        This connector can handle async queries

        """
        def __init__(self, tns_admin_path):
            """
            Args:
                tns_admin_path (str): Oracle driver path
                credentials_path_db1 (str): Database 1 credentials path
                credentials_path_db1 (str): Databsae 2 credentials path
            """
            os.environ['TNS_ADMIN'] = tns_admin_path

            project_root = os.getcwd()
            
            # Load private key
            key_key_path = 'env_admin\key.key'
            with open(key_key_path, "rb") as key_file:
                key = key_file.read()

            cipher = Fernet(key)
            
            # Load encrypted data
            encoded_env_path = r'env_admin\.env.enc'
            with open(encoded_env_path, "rb") as encrypted_file:
                encrypted_data = encrypted_file.read()

            decrypted_data = cipher.decrypt(encrypted_data).decode()

            env_file = StringIO(decrypted_data)
            load_dotenv(stream=env_file)

            self.username1 = os.getenv('main_username')
            self.password1 = os.getenv('main_password')
            self.dsn1 = os.getenv('main_dsn')

            self.username2 = os.getenv('datamart_username')
            self.password2 = os.getenv('datamart_password')
            self.dsn2 = os.getenv('datamart_dsn')

            self.connection_string1 = f'oracle+oracledb://{self.username1}:{self.password1}@{self.dsn1}'
            self.connection_string2 = f'oracle+oracledb://{self.username2}:{self.password2}@{self.dsn2}'

            self.engine1 = create_async_engine(self.connection_string1, max_identifier_length=128, echo=False, future=True)
            self.engine1.dialect.hide_parameters = True
            self.engine2 = create_async_engine(self.connection_string2, max_identifier_length=128, echo=False, future=True)
            self.engine1.dialect.hide_parameters = True


        async def query(self, sql_query, engine=1):
            """
            This allows abstraction of the connection and the class
            so the developer can query a single table as a dataframe

            Args:
                sql_query (str): The query to SQL database is passed as a string
                engine (int): This selects the database. There are two engines:
                    1 -> R1625
                    2 -> COCC DataMart

            Returns:
                df: The SQL query is returned as a pandas DataFrame

            Usage:
                df = db_handler.query("SELECT * FROM DB.TABLE", engine=1)

                In this example, db_handler = DatabaseHandler(args)
            """
            if engine == 1:
                selected_engine = self.engine1
            elif engine == 2:
                selected_engine = self.engine2
            else:
                raise ValueError("Engine must be 1 or 2")

            async with selected_engine.connect() as connection:
                result = await connection.execute(sql_query)
                rows = result.fetchall()
                if not rows:
                    return pd.DataFrame()
                df = pd.DataFrame(rows, columns=result.keys())
            return df

        async def close(self):
            if self.engine1:
                await self.engine1.dispose()
            if self.engine2:
                await self.engine2.dispose()


    # Database Connection Configuration
    tns_admin_path = r'env_admin\tns_admin'
    db_handler = DatabaseHandler(tns_admin_path)

    async def fetch_data(queries):
        try:
            tasks = {query['key']: asyncio.create_task(db_handler.query(query['sql'], query['engine'])) for query in queries}
            results = await asyncio.gather(*tasks.values())
            return {key: df for key, df in zip(tasks.keys(), results)}
        except Exception as e:
            print(f"Error")
            raise
        finally:
            await db_handler.close()

    def run_sql_queries(queries):

        async def run_queries():
            return await fetch_data(queries)
        
        loop = asyncio.get_event_loop()
        if loop.is_running():
            return loop.run_until_complete(run_queries())
        else:
            return asyncio.run(run_queries())
        
    data = run_sql_queries(queries)
    
    return data

In [3]:
# lookup table
# Engine 1
lookup_df = text("""
SELECT 
    *
FROM 
    sys.all_tab_columns col
""")

queries = [
    # {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
    {'key':'lookup_df', 'sql':lookup_df, 'engine':1},
]

data = retrieve_data(queries)
lookup_df = data['lookup_df'].copy()

In [4]:
lookup_df

,owner,table_name,column_name,data_type,data_type_mod,data_type_owner,data_length,data_precision,data_scale,nullable,...,char_used,v80_fmt_image,data_upgraded,histogram,default_on_null,identity_column,evaluation_edition,unusable_before,unusable_beginning,collation
0,SYS,DUAL,DUMMY,VARCHAR2,None,None,1,NaN,NaN,Y,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
1,SYS,SYSTEM_PRIVILEGE_MAP,PRIVILEGE,NUMBER,None,None,22,NaN,NaN,N,...,None,NO,YES,NONE,NO,NO,None,None,None,None
2,SYS,SYSTEM_PRIVILEGE_MAP,NAME,VARCHAR2,None,None,40,NaN,NaN,N,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
3,SYS,SYSTEM_PRIVILEGE_MAP,PROPERTY,NUMBER,None,None,22,NaN,NaN,N,...,None,NO,YES,FREQUENCY,NO,NO,None,None,None,None
4,SYS,TABLE_PRIVILEGE_MAP,PRIVILEGE,NUMBER,None,None,22,NaN,NaN,N,...,None,NO,YES,NONE,NO,NO,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46504,SYS,USER_OGG_AUTO_CAPTURED_TABLES,ALLOW_NOVALIDATE_PK,VARCHAR2,None,None,3,NaN,NaN,Y,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
46505,SYS,USER_OGG_AUTO_CAPTURED_TABLES,NAME,VARCHAR2,None,None,128,NaN,NaN,N,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
46506,SYS,WRR_ROOT_REPLAY_CALL_FILTER,CALL_COUNTER_END,NUMBER,None,None,22,NaN,NaN,N,...,None,NO,YES,NONE,NO,NO,None,None,None,None
46507,SYS,WRR_ROOT_REPLAY_CALL_FILTER,CALL_COUNTER_BEGIN,NUMBER,None,None,22,NaN,NaN,N,...,None,NO,YES,NONE,NO,NO,None,None,None,None


In [ ]:
# Engine 1
acctsubacct = text("""
SELECT 
    a.ACCTNBR,
    a.BALCATCD,
    a.ORIGBAL
FROM 
    OSIBANK.ACCTSUBACCT a
WHERE
    a.BALTYPCD IN ('DLR') AND
    a.BALCATCD IN ('DLRF','DLRS')
""")

queries = [
    # {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
    {'key':'acctsubacct', 'sql':acctsubacct, 'engine':1},
]

data = retrieve_data(queries)
acctsubacct = data['acctsubacct'].copy()

In [9]:
acctsubacct

,acctnbr,subacctnbr,balcatcd,baltypcd,nextrcvbnbr,nextallotnbr,datelastmaint,nextearningdate,origbal,nextratechangedate,earningscalperiodcd,oddfreqnextduedate,nextpayablenbr,partagreenbr,nextaccrualdate,accrualcalperiodcd
0,150708728,5,DLRS,DLR,1,1,2023-01-31 21:54:26,2023-02-28,-2435.04,None,None,None,None,None,None,None
1,150708786,5,DLRF,DLR,1,1,2025-02-28 20:39:37,2025-03-31,-350,None,None,None,None,None,None,None
2,150707928,5,DLRF,DLR,1,1,2022-04-29 21:50:23,2022-05-31,-625,None,None,None,None,None,None,None
3,150864869,5,DLRS,DLR,1,1,2025-02-28 20:40:04,2025-03-31,-1119.36,None,None,None,None,None,None,None
4,150438367,5,DLRF,DLR,1,1,2024-07-31 21:41:11,2024-08-31,-425,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51140,151120228,5,DLRS,DLR,1,1,2025-02-28 20:40:32,2025-03-31,-2450.4,None,None,None,None,None,None,None
51141,151123397,5,DLRF,DLR,1,1,2025-02-28 20:40:32,2025-03-31,-350,None,None,None,None,None,None,None
51142,151124519,5,DLRF,DLR,1,1,2025-02-28 20:40:32,2025-03-31,-625,None,None,None,None,None,None,None
51143,151130764,5,DLRF,DLR,1,1,2025-02-28 20:40:33,2025-03-31,-800,None,None,None,None,None,None,None


In [ ]:
# lookup table
# Engine 1
acctcommon = text("""
SELECT 
    *
FROM 
    OSIBANK.PERS
""")

queries = [
    {'key':'acctcommon', 'sql':acctcommon, 'engine':1},
    # {'key':'wh_pers', 'sql':wh_pers, 'engine':1},
    # {'key':'loans', 'sql':loans, 'engine':1},
    # {'key':'househldacct', 'sql':househldacct, 'engine':1},
    # {'key':'allroles', 'sql':allroles, 'engine':1},
    # {'key':'persaddruse', 'sql':persaddruse, 'engine':1},
    # {'key':'wh_addr', 'sql':wh_addr, 'engine':1},
    # {'key':'pers', 'sql':pers, 'engine':1},
    # {'key':'acctstatistichist', 'sql':acctstatistichist, 'engine':1},
    # {'key':'acctloanlimithist', 'sql':acctloanlimithist, 'engine':1},
]

data = retrieve_data(queries)
pers = data['pers'].copy()



In [5]:
pers

,persnbr,spousepersnbr,restaxctrycd,crcd,salucd,lastname,lastnamesndx,firstname,firstnamesndx,mdlinit,...,firstnameupper,rpt1042syn,graddate,creditreportconsinfocd,privacyyn,purgeyn,shortname,prefculturecd,secondsurname,deathnotificationdate
0,127,NaN,USA,None,None,COCC,C200,DEPOSIT8,D123,None,...,DEPOSIT8,N,NaT,None,N,N,None,None,None,NaT
1,180,NaN,USA,None,None,COCC,C200,ROCH2,R200,None,...,ROCH2,N,NaT,None,N,N,None,None,None,NaT
2,181,NaN,USA,None,None,COCC,C200,ROCH3,R200,None,...,ROCH3,N,NaT,None,N,N,None,None,None,NaT
3,182,NaN,USA,None,None,COCC,C200,ROCH4,R200,None,...,ROCH4,N,NaT,None,N,N,None,None,None,NaT
4,184,NaN,USA,None,None,COCC,C200,PRINT6,P653,None,...,PRINT6,N,NaT,None,N,N,None,None,None,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163959,1167967,NaN,USA,None,None,RAMDHANY CORREA,R535,JESSIE,J200,None,...,JESSIE,N,NaT,None,N,N,None,None,None,NaT
163960,1167973,NaN,USA,None,None,MURPHY,M610,THOMAS,T520,G,...,THOMAS,N,NaT,None,N,N,None,None,None,NaT
163961,1167983,NaN,USA,None,None,COLON TORRES,C453,VICTOR,V236,M,...,VICTOR,N,NaT,None,N,N,None,None,None,NaT
163962,1167985,NaN,USA,None,None,DEFONTES,D153,JOHN,J500,R,...,JOHN,N,NaT,None,N,N,None,None,None,NaT


In [3]:
acctloan = text("""
SELECT 
    *
FROM 
    OSIBANK.WH_ACCTLOAN
""")

queries = [
    {'key':'acctloan', 'sql':acctloan, 'engine':1},
    # {'key':'wh_pers', 'sql':wh_pers, 'engine':1},
    # {'key':'loans', 'sql':loans, 'engine':1},
    # {'key':'househldacct', 'sql':househldacct, 'engine':1},
    # {'key':'allroles', 'sql':allroles, 'engine':1},
    # {'key':'persaddruse', 'sql':persaddruse, 'engine':1},
    # {'key':'wh_addr', 'sql':wh_addr, 'engine':1},
    # {'key':'pers', 'sql':pers, 'engine':1},
    # {'key':'acctstatistichist', 'sql':acctstatistichist, 'engine':1},
    # {'key':'acctloanlimithist', 'sql':acctloanlimithist, 'engine':1},
]

data = retrieve_data(queries)
acctloan = data['acctloan'].copy()

In [ ]:
acctloan

In [3]:
lookup_df[lookup_df['column_name'].str.contains('credlimitclat',case=False,na=False)]

,owner,table_name,column_name,data_type,data_type_mod,data_type_owner,data_length,data_precision,data_scale,nullable,...,char_used,v80_fmt_image,data_upgraded,histogram,default_on_null,identity_column,evaluation_edition,unusable_before,unusable_beginning,collation
1629,COCCDM,WH_ACCTLOAN,CREDLIMITCLATRESEFFDATE,DATE,None,None,7,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
1630,COCCDM,WH_ACCTLOAN,CREDLIMITCLATRESAMT,NUMBER,None,None,22,22.0,3.0,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
1748,COCCDM,WH_ACCTLOAN_TEMP,CREDLIMITCLATRESEFFDATE,DATE,None,None,7,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
1749,COCCDM,WH_ACCTLOAN_TEMP,CREDLIMITCLATRESAMT,NUMBER,None,None,22,22.0,3.0,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
5621,COCCDM,WH_ACCTLOAN_ME,CREDLIMITCLATRESEFFDATE,DATE,None,None,7,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
5622,COCCDM,WH_ACCTLOAN_ME,CREDLIMITCLATRESAMT,NUMBER,None,None,22,22.0,3.0,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None


In [4]:
prop_tables = lookup_df[lookup_df['table_name'].str.contains('prop',case=False,na=False)]

In [6]:
prop_tables['table_name'].unique()

array(['ACCTPROPINS', 'AU_XFR_ENTCHANGESETPROPERTY_V',
       'AU_XFR_ENTITYCHANGESETPROPERTY', 'AU_XFR_ENTITYPROPERTY',
       'PROPUSERFIELD', 'WH_PROP', 'WH_PROP2', 'WH_PROPUSERFIELDS',
       'VIEW_WH_PROPUSERFIELDS', 'ALL_PROPAGATION', 'KU$_MVPROP_VIEW',
       'ALL_METADATA_PROPERTIES', 'KU$_REFPARTTABPROP_VIEW',
       'CDB_DBFS_HS_FIXED_PROPERTIES', 'DATABASE_PROPERTIES',
       'USER_METADATA_PROPERTIES', 'USER_DBFS_HS_FIXED_PROPERTIES',
       'KU$_TABPROP_VIEW', 'KU$_MVLPROP_VIEW', 'KU$_PFHTABPROP_VIEW',
       'DBA_DBFS_HS_FIXED_PROPERTIES', 'KU$_VIEWPROP_VIEW',
       'USER_DBFS_HS_PROPERTIES', 'KU$_MZPROP_VIEW'], dtype=object)

In [7]:
prop_tables[prop_tables['table_name'] == 'ACCTPROPINS']

,owner,table_name,column_name,data_type,data_type_mod,data_type_owner,data_length,data_precision,data_scale,nullable,...,char_used,v80_fmt_image,data_upgraded,histogram,default_on_null,identity_column,evaluation_edition,unusable_before,unusable_beginning,collation
88,COCCDM,ACCTPROPINS,PROPNBR,NUMBER,None,None,22,22.0,0.0,N,...,None,NO,YES,HEIGHT BALANCED,NO,NO,None,None,None,None
89,COCCDM,ACCTPROPINS,LENDERFUNDEDYN,CHAR,None,None,1,NaN,NaN,N,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
90,COCCDM,ACCTPROPINS,INTRPOLICYNBR,NUMBER,None,None,22,22.0,0.0,N,...,None,NO,YES,NONE,NO,NO,None,None,None,None
91,COCCDM,ACCTPROPINS,INACTIVEDATE,DATE,None,None,7,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
92,COCCDM,ACCTPROPINS,ESCROWYN,CHAR,None,None,1,NaN,NaN,N,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
93,COCCDM,ACCTPROPINS,EFFDATE,DATE,None,None,7,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
94,COCCDM,ACCTPROPINS,DATELASTMAINT,DATE,None,None,7,NaN,NaN,N,...,None,NO,YES,NONE,NO,NO,None,None,None,None
95,COCCDM,ACCTPROPINS,ACCTNBR,NUMBER,None,None,22,22.0,0.0,N,...,None,NO,YES,NONE,NO,NO,None,None,None,None


In [11]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent.parent))
import main

data = main.fetch_data()
file_path = Path(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output')


ModuleNotFoundError: No module named 'main'

In [12]:
os.getcwd()

'z:\\Chad Projects\\Monthly Reports\\Automated Linda Reports\\NonOwnerOcc\\Production\\src\\data_connection'